In [7]:
import numpy as np
import string
import pandas as pd

from time import time
from collections import defaultdict
from WordSoupState import WordSoup_state

pd.options.mode.chained_assignment = None  # default='warn'

dictionary_df = pd.read_csv("dictionary_df1.csv")
dictionary_df.loc[160413, "word"] = "null"
dictionary_df.loc[154353, "word"] = "nan"
dictionary_df = dictionary_df[dictionary_df.columns[1:]]

directions = np.array([
    (0, 1), (0, -1), (1, 0), (-1, 0),
    (-1, -1), (-1, 1), (1, -1), (1, 1)
])

# Load the dictionary file
dictionary_set = set(dictionary_df["word"].dropna().astype(str).values)

# Build a prefix set for early stopping in DFS
prefix_set = set()
for word in dictionary_set:
    for i in range(1, len(word) + 1):
        prefix_set.add(word[:i])

score_dict = dict(zip(dictionary_df["word"], dictionary_df["score"]))

keys = list(string.ascii_lowercase)
score = [1,3,3,2,1,4,2,4,3,8,5,5,3,2,1,3,10,1,1,1,2,4,4,8,4,10]
s_dict = dict(zip(keys, score))
def score_func(x):
    return sum(np.vectorize(s_dict.get)(x))

def tiles_remove(tiles, board):
    board[tiles[:, 0], tiles[:, 1]] = "None"
    return board

def gravity(board_1):
    # Gravity down
    row_index, col_index = np.where(board_1 == "None")
    indices = list(zip(row_index, col_index))
    for r, c in indices:
        column = board_1[:r + 1, c].copy()
        if r > 0:
            column[1:] = column[:-1]
            column[0] = "None"
        board_1[:r + 1, c] = column
    # Gravity right
    row_index, col_index = np.where(board_1 == "None")
    indices = list(zip(row_index, col_index))
    for r, c in indices:
        row = board_1[r, :c + 1].copy()
        if c > 0:
            row[1:] = row[:-1]
            row[0] = "None"
        board_1[r, :c + 1] = row
    return board_1


def Move(board, word_lst=[], score=0):
    # 1. DFS to find the possible words in the board
    def dfs(x, y, path, visited, word):
        word += board[x, y]
        if word not in prefix_set:
            return  # Stop early if no word starts with the current prefix

        if word in dictionary_set:
            found_words.append([word, path])

        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < board.shape[0] and 0 <= ny < board.shape[1] and (nx, ny) not in visited:
                dfs(nx, ny, path + [(nx, ny)], visited | {(nx, ny)}, word)

    # Find all valid words in the grid
    found_words = []
    for i in range(board.shape[0]):
        for j in range(board.shape[1]):
            dfs(i, j, [(i, j)], {(i, j)}, "")
    if len(found_words) == 0:
        return []

    found_words_df = pd.DataFrame(found_words)
    found_words_df.columns = ["word", "indices"]
    found_words_df["score"] = found_words_df["word"].apply(lambda x: score_dict[x])
    found_words_df.sort_values(by="score", ascending=False, inplace=True)

    # 2. Remove corresponding tiles
    board1 = found_words_df["indices"].apply(lambda x: tiles_remove(np.array(x), board.copy()))

    # 3. Apply gravity to the reduced board
    board1.apply(lambda x: gravity(x))

    # 4. Return (resulted board, word, score)
    def Word(x):
        w_lst = [x["word"]]
        w_lst.extend(word_lst)
        return w_lst

    output = list(zip(board1, found_words_df.apply(Word, axis=1), found_words_df["score"] + score))
    return output


def DFS_Algorithm(board, step=4):
    '''
    Input: board and streamlined dictionary
    Output: (Best score, combination of words)
    '''
    _ = Move(board, [], 0)

    if len(_) == 0:
        return []
    queue = list(zip(_, np.ones(len(_)).astype("int64")))
    word_score_lst = [(i[1], i[2]) for i in _]
    max_score = sorted(word_score_lst, key=lambda x: x[1], reverse=True)[0][1]
    best_ = [i for i in _ if i[2] == max_score]

    n = 1
    if step == 1:
        return best_

    while queue:
        n += 1
        b_tri, index = queue.pop(0)
        b, w, s = b_tri
        strings = b[np.where(b != "None")]

        if index >= step:
            continue
        if len(strings) == 0:
            continue
        if (s + len(strings) * score_func(strings)) <= max_score:
            continue
        board_lst = Move(b, word_lst=w, score=s)
        if index == step - 1:
            x = max_score - s
            board_lst = [i for i in board_lst if i[2] > x]

        if board_lst:
            _ = list(zip(board_lst, np.ones(len(board_lst)) + index))
            bs = sorted(board_lst, key=lambda x: x[2], reverse=True)[0][2]
            if bs > max_score:
                max_score = bs
                print(max_score)
                best_ = [i for i in board_lst if i[2] == max_score]
            queue = _ + queue
        else:
            continue
    return best_

board = np.array([
        ['z', 'h', 's', 's', 'g', 'a', 's', 'p', 'r'],
        ['r', 'q', 'q', 'i', 'm', 'n', 'o', 'd', 'e'],
        ['n', 'b', 'a', 's', 't', 'd', 't', 'i', 'o'],
        ['e', 'a', 't', 'k', 'k', 'n', 'a', 'a', 'd'],
        ['o', 'o', 'r', 'n', 'a', 'y', 'n', 't', 'o'],
        ['n', 'p', 'u', 'l', 'r', 'e', 'a', 'o', 'a'],
        ['t', 'm', 's', 'e', 'm', 'r', 'p', 'n', 'y'],
        ['o', 'n', 'e', 'l', 'a', 's', 'u', 'e', 'a'],
        ['p', 'h', 't', 'q', 'm', 'l', 'u', 'r', 'o'],
        ['i', 'c', 'u', 'o', 'n', 'x', 'o', 'i', 'b'],
        ['o', 'e', 'n', 'i', 'i', 'r', 'n', 'x', 'k'],
        ['i', 'q', 'u', 'a', 'p', 'f', 't', 'l', 'e']
    ], dtype='<U4')

word_lst = []
Score = 0
while True:
    l = DFS_Algorithm(board, step=1)
    if len(l) > 0:
        board, word, s = l[0]
        word_lst+=word
        Score+=s
        if np.sum(board!="None")==0:
            Score+=500
    else:
        print("***************Game Over****************")
        print(f"Score: ----------------------------{Score}")
        break

***************Game Over****************
Score: ----------------------------1834


In [8]:
class GreedyNode:
    def __init__(self, state):
        self.state = state
        self._untried_actions = self.untried_actions()
        return
    
    def untried_actions(self):
        self._untried_actions = self.state.get_legal_actions()
        return self._untried_actions
    
    def best_move(self):
        current_node = self
        if len(current_node._untried_actions) != 0:
            action = sorted(current_node._untried_actions, key=lambda x: score_dict[x[0]], reverse=True)[0]
            return current_node.state.move(action)

In [9]:
class MonteCarloTreeSearchNode():
    def __init__(self, state, parent=None, parent_action=None, Score=1800, sim=100, position=0):
        self.state = state
        self.parent = parent
        self.parent_action = parent_action
        self.children = []
        self._number_of_visits = 0
        self._results = defaultdict(int)
        self._results[1] = 0
        self._results[-1] = 0
        self._untried_actions = None
        self._untried_actions = self.untried_actions()
        self.Score = Score
        self.sim = sim
        self.position = position
        return

    def untried_actions(self):
        self._untried_actions = self.state.get_legal_actions()
        return self._untried_actions

    def q(self):
        wins = self._results[1]
        loses = self._results[-1]
        return wins - loses

    def n(self):
        return self._number_of_visits

    def expand(self):
        possible_moves = [i for i in self._untried_actions if len(i[0])==7]
        action = self.rollout_policy(possible_moves)
        next_state = self.state.move(action)
        child_node = MonteCarloTreeSearchNode(next_state, parent=self, parent_action=action,
                                              Score=self.Score, sim=self.sim, position=self.position+1)

        self.children.append(child_node)
        return child_node

    def is_terminal_node(self):
        return self.state.is_game_over()

    def rollout(self):
        current_rollout_state = self.state

        while not current_rollout_state.is_game_over():
            possible_moves = current_rollout_state.get_legal_actions()
            action = self.rollout_policy(possible_moves)
            current_rollout_state = current_rollout_state.move(action)

        return current_rollout_state.game_result(self.Score)

    def backpropagate(self, result):
        self._number_of_visits += 1.
        self._results[result] += 1.
        if self.parent:
            self.parent.backpropagate(result)

    def is_fully_expanded(self):
        return len(self._untried_actions) == 0

    def best_child(self, c_param=0.1):
        choices_weights = [(c.q() / c.n()) + c_param * np.sqrt((2 * np.log(self.n()) / c.n())) for c in self.children]
        return self.children[np.argmax(choices_weights)]

    def rollout_policy(self, possible_moves):
        return possible_moves[np.random.randint(len(possible_moves))]

    def _tree_policy(self):
        current_node = self
        while not current_node.is_terminal_node():
            if not current_node.is_fully_expanded():
                return current_node.expand()
            else:
                current_node = current_node.best_child()
        return current_node

    def best_action(self):
        simulation_no = self.sim

        for i in range(simulation_no):
            v = self._tree_policy()
            reward = v.rollout()
            v.backpropagate(reward)

        return self.best_child(c_param=0.1)

In [10]:
%%time
board = np.array([
        ['z', 'h', 's', 's', 'g', 'a', 's', 'p', 'r'],
        ['r', 'q', 'q', 'i', 'm', 'n', 'o', 'd', 'e'],
        ['n', 'b', 'a', 's', 't', 'd', 't', 'i', 'o'],
        ['e', 'a', 't', 'k', 'k', 'n', 'a', 'a', 'd'],
        ['o', 'o', 'r', 'n', 'a', 'y', 'n', 't', 'o'],
        ['n', 'p', 'u', 'l', 'r', 'e', 'a', 'o', 'a'],
        ['t', 'm', 's', 'e', 'm', 'r', 'p', 'n', 'y'],
        ['o', 'n', 'e', 'l', 'a', 's', 'u', 'e', 'a'],
        ['p', 'h', 't', 'q', 'm', 'l', 'u', 'r', 'o'],
        ['i', 'c', 'u', 'o', 'n', 'x', 'o', 'i', 'b'],
        ['o', 'e', 'n', 'i', 'i', 'r', 'n', 'x', 'k'],
        ['i', 'q', 'u', 'a', 'p', 'f', 't', 'l', 'e']
    ], dtype='<U4')
initial_state = WordSoup_state(board)

t0 = time()
root = MonteCarloTreeSearchNode(state = initial_state, Score=Score, sim=20)
selected_node = root.best_action()
t1 = time()
print(selected_node.state.board)
print(f"Move takes {t1 - t0} s")
print(f"Words taken: {selected_node.state.word_lst}")
print(f"Score so far is {selected_node.state.score}")
print("*"*70)

t0 = time()
root = GreedyNode(selected_node.state)
selected_node = root.best_move()
t1 = time()
print(selected_node.board)
print(f"Move takes {t1 - t0} s")
print(f"Words taken: {selected_node.word_lst}")
print(f"Score so far is {selected_node.score}")
print("*"*70)

while not selected_node.is_game_over():
    t0 = time()
    root = GreedyNode(selected_node)
    selected_node = root.best_move()
    t1 = time()
    print(selected_node.board)
    print(f"Move takes {t1 - t0} s")
    print(f"Words taken: {selected_node.word_lst}")
    print(f"Score so far is {selected_node.score}")
    print("*"*70)

print("--------------------Game over------------------------")

[['None' 'None' 'None' 'None' 'z' 'a' 's' 'p' 'r']
 ['None' 'None' 'r' 's' 'g' 'n' 'o' 'd' 'e']
 ['None' 'n' 'h' 'i' 'm' 'd' 't' 'i' 'o']
 ['e' 'q' 's' 's' 't' 'n' 'a' 'a' 'd']
 ['o' 'b' 'q' 'k' 'k' 'y' 'n' 't' 'o']
 ['n' 'a' 'a' 'n' 'a' 'e' 'a' 'o' 'a']
 ['t' 'o' 't' 'l' 'r' 'r' 'p' 'n' 'y']
 ['o' 'p' 'r' 'e' 'm' 's' 'u' 'e' 'a']
 ['p' 'm' 'u' 'l' 'a' 'l' 'u' 'r' 'o']
 ['i' 'n' 't' 'q' 'n' 'x' 'o' 'i' 'b']
 ['o' 'e' 'n' 'i' 'i' 'r' 'n' 'x' 'k']
 ['i' 'q' 'u' 'a' 'p' 'f' 't' 'l' 'e']]
Move takes 60.59141778945923 s
Words taken: ['mouches']
Score so far is 105
**********************************************************************
[['None' 'None' 'None' 'None' 'z' 'a' 's' 'p' 'r']
 ['None' 'None' 'None' 's' 'g' 'n' 'o' 'd' 'e']
 ['None' 'None' 'None' 'i' 'm' 'd' 't' 'i' 'o']
 ['None' 'None' 'None' 's' 't' 'n' 'a' 'a' 'd']
 ['None' 'None' 'r' 'k' 'k' 'y' 'n' 't' 'o']
 ['None' 'None' 'h' 'n' 'a' 'e' 'a' 'o' 'a']
 ['None' 'e' 's' 'l' 'r' 'r' 'p' 'n' 'y']
 ['o' 'n' 'q' 'e' 'm' 's' 'u' 'e' 'a

In [11]:
def GreedyExhaustive(board):
    initial_state = WordSoup_state(board)
    t0 = time()
    root = GreedyNode(initial_state)
    selected_node = root.best_move()
    t1 = time()
    print(selected_node.board)
    print(f"Move takes {t1 - t0} s")
    print(f"Words taken: {selected_node.word_lst}")
    print(f"Score so far is {selected_node.score}")
    print("*"*70)

    while not selected_node.is_game_over():
        if np.sum(selected_node.board!="None") > 15:
            t0 = time()
            root = GreedyNode(selected_node)
            selected_node = root.best_move()
            t1 = time()
            print(selected_node.board)
            print(f"Move takes {t1 - t0} s")
            print(f"Words taken: {selected_node.word_lst}")
            print(f"Score so far is {selected_node.score}")
            print("*"*70)
        else:
            t0 = time()
            l = DFS_Algorithm(selected_node.board, step=5)
            B, _, S = l[0]
            S += selected_node.score
            if np.sum(B!="None")==0:
                S+=500
            t1 = time()
            print(B)
            print(f"Move takes {t1 - t0} s")
            print(f"Words taken: {selected_node.word_lst+_}")
            print(f"Score so far is {S}")
            print("*"*70)
            break
            

    print("--------------------Game over------------------------")

In [12]:
board = np.array([
        ['z', 'h', 's', 's', 'g', 'a', 's', 'p', 'r'],
        ['r', 'q', 'q', 'i', 'm', 'n', 'o', 'd', 'e'],
        ['n', 'b', 'a', 's', 't', 'd', 't', 'i', 'o'],
        ['e', 'a', 't', 'k', 'k', 'n', 'a', 'a', 'd'],
        ['o', 'o', 'r', 'n', 'a', 'y', 'n', 't', 'o'],
        ['n', 'p', 'u', 'l', 'r', 'e', 'a', 'o', 'a'],
        ['t', 'm', 's', 'e', 'm', 'r', 'p', 'n', 'y'],
        ['o', 'n', 'e', 'l', 'a', 's', 'u', 'e', 'a'],
        ['p', 'h', 't', 'q', 'm', 'l', 'u', 'r', 'o'],
        ['i', 'c', 'u', 'o', 'n', 'x', 'o', 'i', 'b'],
        ['o', 'e', 'n', 'i', 'i', 'r', 'n', 'x', 'k'],
        ['i', 'q', 'u', 'a', 'p', 'f', 't', 'l', 'e']
    ], dtype='<U4')
GreedyExhaustive(board)

[['None' 'None' 'None' 'None' 'None' 'z' 'h' 's' 'r']
 ['None' 'None' 'None' 'None' 'r' 'q' 'q' 's' 'e']
 ['None' 'None' 'n' 'b' 'a' 'i' 'g' 'a' 'o']
 ['None' 'e' 'a' 't' 's' 'm' 'd' 'p' 'd']
 ['o' 'o' 'r' 'k' 't' 'n' 'a' 'd' 'o']
 ['n' 'p' 'u' 'n' 'k' 'y' 'a' 'o' 'a']
 ['t' 'm' 's' 'l' 'a' 'r' 'p' 'n' 'y']
 ['o' 'n' 'e' 'e' 'r' 's' 'u' 'e' 'a']
 ['p' 'h' 't' 'q' 'm' 'l' 'u' 'r' 'o']
 ['i' 'c' 'u' 'o' 'n' 'x' 'o' 'i' 'b']
 ['o' 'e' 'n' 'i' 'i' 'r' 'n' 'x' 'k']
 ['i' 'q' 'u' 'a' 'p' 'f' 't' 'l' 'e']]
Move takes 0.24414992332458496 s
Words taken: ['lamentations']
Score so far is 264
**********************************************************************
[['None' 'None' 'None' 'None' 'None' 'z' 'h' 's' 'r']
 ['None' 'None' 'None' 'None' 'r' 'q' 'q' 's' 'e']
 ['None' 'None' 'None' 'None' 'a' 'i' 'g' 'a' 'o']
 ['None' 'None' 'None' 'None' 's' 'm' 'd' 'p' 'd']
 ['None' 'None' 'o' 'b' 't' 'n' 'a' 'd' 'o']
 ['None' 'n' 'e' 't' 'k' 'y' 'a' 'o' 'a']
 ['t' 'o' 'n' 'k' 'a' 'r' 'p' 'n' 'y']
 ['o' 'p

[['None' 'None' 'None' 'None' 'None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None' 'None' 'None' 'None' 'o']
 ['None' 'None' 'None' 'None' 'None' 'None' 'None' 'm' 'q']
 ['None' 'None' 'o' 'p' 'r' 'n' 'g' 'q' 'z']
 ['i' 'e' 'o' 'n' 'n' 'x' 'p' 'x' 'r']]
Move takes 0.00449681282043457 s
Words taken: ['lamentations', 'quinches', 'quaternary', 'unmanlike', 'anaerobiont', 'slapdash', 'odourful', 'akimbo', 'tiptop', 'todays', 'rekes']
Score so far is 1676
***************************************************************

In [ ]:
%%time
DFS_Algorithm(board)